# 02 - Preparing the Dataset

On this stage we'll fix inconsisteces and missing values

In [1]:
import pandas as pd
import numpy as np
import struct as st
import socket as sk
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('../Dataset/UNSW-NB15.csv', index_col = [0])

C:\Users\lipey\AppData\Local\Temp\ipykernel_1740\32101263.py:1: DtypeWarning: Columns (2,4,40,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Dataset/UNSW-NB15.csv', index_col = [0])


### Droping unnecessary column

In [3]:
df.drop(columns='attack_cat', inplace = True)

### Fixing the Router's Address

In [4]:
df.loc[df.srcip == '192168241243','srcip'] = '192.168.241.243'
df.loc[df.dstip == '192168241243','dstip'] = '192.168.241.243'

### Handling inconsistent data

In [5]:
# Converting IP to INT
df['srcip'] = df.srcip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])
df['dstip'] = df.dstip.apply(lambda x: st.unpack('!I', sk.inet_aton(x))[0])

In [6]:
# Converting HEX to DEC
df.loc[df.dsport == '0xc0a8','dsport'] = '49320'
df.loc[df.dsport == '0x20205321','dsport'] = '538989345'
df.loc[df.dsport == '0xcc09','dsport'] = '52233'
df.loc[df.sport == '0x000b','sport'] = '11'
df.loc[df.sport == '0x000c','sport'] = '12'

In [7]:
# Inconsistence and  Missing values
df.loc[df.ct_ftp_cmd == ' ', 'ct_ftp_cmd'] = df.ct_flw_http_mthd.mean()
df.drop(df.loc[df.dsport == '-'].index, inplace = True)
df.drop(df.loc[df.sport == '-'].index, inplace = True)
df.ct_flw_http_mthd.fillna(df.ct_flw_http_mthd.mean(), inplace = True)
df.is_ftp_login.fillna(df.is_ftp_login .mean(), inplace = True)

In [8]:
# Converting object
df.sport.astype(int)
df.dsport.astype(int)
df.ackdat.astype(float)
df.dur.astype(float)

0         0.001055
1         0.036133
2         0.001119
3         0.001209
4         0.001169
            ...   
440039    0.087306
440040    0.365058
440041    6.335154
440042    2.200934
440043    0.942984
Name: dur, Length: 2540011, dtype: float64

In [9]:
# Handling data
Encoder = LabelEncoder()
df.proto = Encoder.fit_transform(df.proto)
df.state = Encoder.fit_transform(df.state)
df.service = Encoder.fit_transform(df.service)

In [10]:
# Handling NaN Values
df.dur.fillna(df.dur.mean(), inplace = True)
df.ackdat.fillna(df.ackdat.mean(), inplace = True)

### Exporting

In [11]:
df.to_csv('../Dataset/Dataset-Preprocessing.csv')